In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import joblib

In [ ]:
adata_file = "/ocean/projects/cis240075p/asachan/datasets/B_Cell/T_cell/outs/stream_objs/rbpj_ntc/rbpj_ntc_v2.pkl"
adata = joblib.load(adata_file)